In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import math
from scipy.interpolate import UnivariateSpline
import pickle
%matplotlib inline
import dill
import warnings

warnings.filterwarnings("ignore",category=DeprecationWarning)
dill.settings['recurse']=True

In [15]:
# play with 2013 subset
f=open('../../python_code/loans_df','r')
loans = pickle.load(f)
f.close()
loans = loans.loc[loans['year']<=2012]

loans.shape

(92964, 82)

(55546, 82)

In [7]:
from sklearn import preprocessing
def encodeColumns(df,colNames):
    encs ={}
    for i in colNames:
        enc=preprocessing.LabelEncoder()
        enc.fit(df[i])
        df["enc_"+i]=enc.transform(df[i])
        encs[i] = enc
    return df, encs
    
#encode verification, homeOwnership, purpose
toEncode = ['verification_status','home_ownership','purpose','grade','sub_grade','Renewable']
df, encodings = encodeColumns(loans,toEncode)

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
scaler = StandardScaler()

#[float(l[:-1])/100 for l in loans.revol_util]
loans.revol_util=loans.revol_util.apply(lambda x:float(str(x).strip('%')))
loans= loans[np.isfinite(loans.revol_util)]
X=np.array(loans[['enc_grade','enc_sub_grade','dti','revol_util']])

LR = LinearRegression()
loans['total_rec_prct']=loans.total_pymnt_inv/loans.funded_amnt
y=np.array(loans.total_rec_prct)

pipeline = Pipeline([('scaler',StandardScaler()),
                    ('regressor',LinearRegression())])
pipeline.fit(X,y)
pipeline.predict(X[1])[0]

f= open('simpleLoanModelLR','wb')
pickle.dump(pipeline,f)
f.close()
f=open('dillModelsAndEncoders','wb')
dill.dump({'pipeline':pipeline,'encodings':encodings},f)
f.close()

In [9]:
f=open('dillModelsAndEncoders','r')
dct= dill.load(f)
mdl_=dct['pipeline']
encodings=dct['encodings']
grade = 'A'
subGrade = '2'
x=[encodings['grade'].transform(grade),encodings['sub_grade'].transform(grade+subGrade),1,70]
ret=mdl_.predict(x)[0]-1
prc_return="{0:.2f}".format(ret*100)+'%'
print prc_return

12.08%


/Users/Mikebo/miniconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:646: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/Mikebo/miniconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [14]:
np.unique(loans.term)

array([' 36 months', ' 60 months'], dtype=object)